# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846830353910                   -0.70    4.8
  2   -7.852313988077       -2.26       -1.53    1.0
  3   -7.852646157169       -3.48       -2.52    3.0
  4   -7.852646676178       -6.28       -3.35    2.5
  5   -7.852646685884       -8.01       -4.74    1.5
  6   -7.852646686724       -9.08       -5.23    3.5
  7   -7.852646686730      -11.27       -6.05    1.0
  8   -7.852646686730      -12.50       -7.34    3.0
  9   -7.852646686730   +  -15.05       -7.55    2.5
 10   -7.852646686730      -15.05       -8.52    2.0
 11   -7.852646686730      -14.45       -9.07    2.2
 12   -7.852646686730   +  -14.75      -10.02    1.8
 13   -7.852646686730   +    -Inf      -10.82    2.8
 14   -7.852646686730   +  -14.45      -11.03    1.0
 15   -7.852646686730      -14.75      -12.34    1.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846791221301                   -0.70           4.5
  2   -7.852552419987       -2.24       -1.63   0.80    2.2
  3   -7.852640037770       -4.06       -2.72   0.80    1.0
  4   -7.852646498475       -5.19       -3.39   0.80    1.8
  5   -7.852646683305       -6.73       -4.31   0.80    2.0
  6   -7.852646686599       -8.48       -4.91   0.80    2.0
  7   -7.852646686719       -9.92       -5.80   0.80    1.5
  8   -7.852646686730      -10.98       -6.32   0.80    2.5
  9   -7.852646686730      -13.02       -7.26   0.80    1.0
 10   -7.852646686730      -13.97       -7.57   0.80    2.2
 11   -7.852646686730      -14.45       -8.69   0.80    1.2
 12   -7.852646686730   +    -Inf       -9.30   0.80    2.2
 13   -7.852646686730   +    -Inf       -9.96   0.80    1.8
 14   -7.852646686730   +  -14.75      -11.25   0.80    1.5
 15   -7.852646686730      -14.27      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.368744e+01     3.314661e+00
 * time: 0.3522648811340332
     1     8.739886e-01     2.084213e+00
 * time: 0.5442209243774414
     2    -1.649596e+00     2.482197e+00
 * time: 0.5656239986419678
     3    -3.627294e+00     1.851537e+00
 * time: 0.5966458320617676
     4    -4.763604e+00     1.624632e+00
 * time: 0.6276838779449463
     5    -6.589874e+00     1.093612e+00
 * time: 0.6602458953857422
     6    -7.367496e+00     5.728853e-01
 * time: 0.6918418407440186
     7    -7.661178e+00     1.546498e-01
 * time: 0.7136399745941162
     8    -7.755698e+00     1.109593e-01
 * time: 0.73590087890625
     9    -7.804360e+00     7.468496e-02
 * time: 0.7571179866790771
    10    -7.827254e+00     6.024873e-02
 * time: 0.7784547805786133
    11    -7.842160e+00     5.460403e-02
 * time: 0.8010118007659912
    12    -7.847950e+00     3.479827e-02
 * time: 0.8229429721832275
    13    -7.850656e+00     1.676373e-02
 * time: 0.845388889312

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846787289191                   -0.70    4.2
  2   -7.852282259302       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685753                   -2.54
  2   -7.852646686730       -9.01       -5.52
  3   -7.852646686730      -14.75      -10.20
  4   -7.852646686730   +  -14.75      -15.40


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.065528023912874e-13
|ρ_newton - ρ_scfv| = 7.922038055545094e-13
|ρ_newton - ρ_dm|   = 2.620329871897311e-10
